## Import

In [2]:
pip install dash

   ---------------------------------------- 0.0/10.2 MB ? eta -:--:--
   ---------------------------------------- 0.1/10.2 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.3/10.2 MB 3.4 MB/s eta 0:00:03
   - -------------------------------------- 0.5/10.2 MB 3.9 MB/s eta 0:00:03
   - -------------------------------------- 0.5/10.2 MB 3.9 MB/s eta 0:00:03
   ---- ----------------------------------- 1.1/10.2 MB 5.0 MB/s eta 0:00:02
   ----- ---------------------------------- 1.4/10.2 MB 5.2 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/10.2 MB 5.1 MB/s eta 0:00:02
   ------- -------------------------------- 1.9/10.2 MB 5.1 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/10.2 MB 5.2 MB/s eta 0:00:02
   --------- ------------------------------ 2.3/10.2 MB 5.1 MB/s eta 0:00:02
   ---------- ----------------------------- 2.7/10.2 MB 5.4 MB/s eta 0:00:02
   ----------- ---------------------------- 2.9/10.2 MB 5.5 MB/s eta 0:00:02
   ---

In [3]:
pip install plotly

   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/15.6 MB 330.3 kB/s eta 0:00:48
   ---------------------------------------- 0.1/15.6 MB 409.6 kB/s eta 0:00:39
   ---------------------------------------- 0.2/15.6 MB 919.0 kB/s eta 0:00:17
    --------------------------------------- 0.3/15.6 MB 1.4 MB/s eta 0:00:11
    --------------------------------------- 0.3/15.6 MB 1.4 MB/s eta 0:00:12
   - -------------------------------------- 0.4/15.6 MB 1.4 MB/s eta 0:00:12
   - -------------------------------------- 0.5/15.6 MB 1.5 MB/s eta 0:00:11
   - -------------------------------------- 0.6/15.6 MB 1.5 MB/s eta 0:00:10
   - -------------------------------------- 0.7/15.6 MB 1.5 MB/s eta 0:00:10
   -- ------------------------------------- 0.8/15.6 MB 1.7 MB/s eta 0:00:09
   -- --------

In [3]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import dash
from dash import dcc
import dash_core_components as dcc
from dash import html
import dash_html_components as html
from dash.dependencies import Input, Output

In [4]:
print(dash.__version__)

2.14.2


In [5]:
app = dash.Dash(__name__)

In [6]:
# -- Import and clean data (importing csv into pandas)
# df = pd.read_csv("intro_bees.csv")
df = pd.read_csv("https://raw.githubusercontent.com/Coding-with-Adam/Dash-by-Plotly/master/Other/Dash_Introduction/intro_bees.csv")

df

Program  Year        Period    State  ANSI        Affected by  \
0     SURVEY  2019  JAN THRU MAR  Alabama     1            Disease   
1     SURVEY  2019  JAN THRU MAR  Alabama     1              Other   
2     SURVEY  2019  JAN THRU MAR  Alabama     1         Pesticides   
3     SURVEY  2019  JAN THRU MAR  Alabama     1  Pests_excl_Varroa   
4     SURVEY  2019  JAN THRU MAR  Alabama     1            Unknown   
...      ...   ...           ...      ...   ...                ...   
4585  SURVEY  2015  OCT THRU DEC  Wyoming    56              Other   
4586  SURVEY  2015  OCT THRU DEC  Wyoming    56         Pesticides   
4587  SURVEY  2015  OCT THRU DEC  Wyoming    56  Pests_excl_Varroa   
4588  SURVEY  2015  OCT THRU DEC  Wyoming    56            Unknown   
4589  SURVEY  2015  OCT THRU DEC  Wyoming    56       Varroa_mites   

      Pct of Colonies Impacted state_code  
0                          1.8         AL  
1                          3.1         AL  
2                          0.3         AL  
3                         22.7         AL  
4                          9.0         AL  
...                        ...        ...  
4585                      11.3         WY  
4586                      10.5         WY  
4587                       3.3         WY  
4588                       3.9         WY  
4589                      45.0         WY  

[4590 rows x 8 columns]

In [7]:
df = df.groupby(['State', 'ANSI', 'Affected by', 'Year', 'state_code'])[['Pct of Colonies Impacted']].mean()
df.reset_index(inplace=True)
df[:5]

State  ANSI Affected by  Year state_code  Pct of Colonies Impacted
0  Alabama     1     Disease  2015         AL                      0.05
1  Alabama     1     Disease  2016         AL                      1.20
2  Alabama     1     Disease  2017         AL                      2.25
3  Alabama     1     Disease  2018         AL                      1.30
4  Alabama     1     Disease  2019         AL                      1.80

In [14]:
# App layout
app.layout = html.Div([

    html.H1("Web Application Dashboards with Dash", style={'text-align': 'center'}),

    dcc.Dropdown(id="slct_year",
                 options=[
                     {"label": "2014", "value": 2014},
                     {"label": "2016", "value": 2016},
                     {"label": "2017", "value": 2017},
                     {"label": "2018", "value": 2018}],
                 multi=False,
                 value=2015,
                 style={'width': "40%"}
                 ),

    html.Div(id='output_container', children=[]),
    html.Br(),

    dcc.Graph(id='my_bee_map', figure={})

])

# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_bee_map', component_property='figure')],
    [Input(component_id='slct_year', component_property='value')]
)
def update_graph(option_slctd):
    print(option_slctd)
    print(type(option_slctd))

    container = "The year chosen by user was: {}".format(option_slctd)

    dff = df.copy()
    dff = dff[dff["Year"] == option_slctd]
    dff = dff[dff["Affected by"] == "Varroa_mites"]

    # Plotly Express
    fig = px.choropleth(
        data_frame=dff,
        locationmode='USA-states',
        locations='state_code',
        scope="usa",
        color='Pct of Colonies Impacted',
        hover_data=['State', 'Pct of Colonies Impacted'],
        color_continuous_scale=px.colors.sequential.YlOrRd,
        labels={'Pct of Colonies Impacted': '% of Bee Colonies'},
        template='plotly_dark'
    )

    # Plotly Graph Objects (GO)
    # fig = go.Figure(
    #     data=[go.Choropleth(
    #         locationmode='USA-states',
    #         locations=dff['state_code'],
    #         z=dff["Pct of Colonies Impacted"].astype(float),
    #         colorscale='Reds',
    #     )]
    # )
    #
    # fig.update_layout(
    #     title_text="Bees Affected by Mites in the USA",
    #     title_xanchor="center",
    #     title_font=dict(size=24),
    #     title_x=0.5,
    #     geo=dict(scope='usa'),
    # )

    return container, fig

if __name__ == '__main__':
    app.run_server()

In [13]:
if __name__ == '__main__':
    app.run_server()